In [1]:
import jax
import pennylane as qml
from pennylane import numpy as np
from pennylane import grad
from pennylane.fourier import circuit_spectrum
import matplotlib

In [2]:
dev = qml.device("default.qubit", wires=n_qubits)

# Quantum Layer: First LQ Embed Layer
def lq_embed_1(n_qubits, theta, theta_par):
    for h in range(n_qubits):
        qml.Hadamard(wires=h)
    for i in range(n_qubits):
        qml.RX(theta[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])
    for i in range(n_qubits):
        qml.RY(theta_par[i], wires=i)

# Quantum Layer: Subsequent Layers
def lq_embed_m(n_qubits, theta, theta_par):
    for i in range(n_qubits):
        qml.RX(theta[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])
    for i in range(n_qubits):
        qml.RY(theta_par[i], wires=i)

# Full Layerwise Ansatz
@qml.qnode(dev, interface="autograd")
def critic_qnode(inputs, weights):
    lq_embed_1(inputs, weights[0])
    for m in range(1, len(weights)):
        lq_embed_m(inputs, weights[m])
    return qml.expval(qml.PauliZ(0))

NameError: name 'n_qubits' is not defined

In [55]:
@qml.qnode(dev)
def actor_qnode(n_qubits, state_input, theta, theta_par):
    # Apply Hadamard
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    
    # RX with state-dependent angle
    for i in range(n_qubits):
        qml.RX(state_input[i], wires=i)
    
    # Entanglement (CZ between neighbours)
    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])
    
    # RY with trainable parameter
    for i in range(0, n_qubits):
        qml.RY(theta_par[i], wires=i)
    
    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [56]:
@qml.qnode(dev)
def critic_qnode(n_qubits, critic_input, theta, theta_par):
    # Build same ansatz as above, with critic_input
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.RX(critic_input[i], wires=i)

    for i in range(n_qubits - 1):
        qml.CZ(wires=[i, i+1])

    for i in range(0, n_qubits):
        qml.RY(theta_par[i], wires=i)

    # Q-value output (e.g., Z₀ expectation)
    return [qml.expval(qml.PauliZ(0)) for i in range(n_qubits)]

In [3]:
def make_critic_qnode(n_qubits):
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def critic_qnode(critic_input, theta, theta_par):
        # Build same ansatz as above, with critic_input
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
            qml.RX(critic_input[i], wires=i)
    
        for i in range(n_qubits - 1):
            qml.CZ(wires=[i, i+1])
    
        for i in range(0, n_qubits):
            qml.RY(theta_par[i], wires=i)
    
        # Q-value output (e.g., Z₀ expectation)
        #return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
        return qml.expval(qml.PauliZ(i))
    return critic_qnode

def make_actor_qnode(n_qubits):
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def actor_qnode(state_input, theta, theta_par):
        # Apply Hadamard
        for i in range(n_qubits):
            qml.Hadamard(wires=i)
        
        # RX with state-dependent angle
        for i in range(n_qubits):
            qml.RX(state_input[i], wires=i)
        
        # Entanglement (CZ between neighbours)
        for i in range(n_qubits - 1):
            qml.CZ(wires=[i, i+1])
        
        # RY with trainable parameter
        for i in range(0, n_qubits):
            qml.RY(theta_par[i], wires=i)
        
        # Measurement
        #return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
        return qml.expval(qml.PauliZ(i))
    return actor_qnode

In [4]:
# TODO: PROPERLY PORT THE QISKIT VERSION TO PENNYLANE
# TODO: IMPORT THE UAV LQDRL GYM ENVIRONMENT TO GET & UPDATE THESE VALUES AFTER EVERY LAYER
m_layers = 3
E_rem = 500e03
num_gus = 4
u = []
uav_pos = [0, 0, 0]
uav_v_max = 30
zeta_v = 1
uav_v = zeta_v * uav_v_max
noma_g = [1, 2, 3]
pwr_alloc = 1
action_vec = []

action_vec.append(uav_v)
for k in range(len(noma_g)):
    action_vec.append(noma_g[k])
action_vec.append(pwr_alloc)

actor_inp_vec = []
crit_inp_vec = []

for i in range(len(uav_pos)):
    actor_inp_vec.append(uav_pos[i])
actor_inp_vec.append(E_rem)
for j in range(2*num_gus):
    u.append(j)
    actor_inp_vec.append(u[j])

full_input = np.concatenate([actor_inp_vec, action_vec])

n_qubits = len(actor_inp_vec)
print(n_qubits) 
n_crit_qubits = len(full_input)
print(n_crit_qubits)

theta = np.zeros(n_qubits, requires_grad=True)
theta_par = np.random.randn(n_qubits, requires_grad=True)

crit_theta = np.zeros(n_crit_qubits, requires_grad=True)
crit_theta_par = np.random.randn(n_crit_qubits, requires_grad=True)

critic_qnode = make_critic_qnode(n_crit_qubits)
#q_val = critic_qnode(n_crit_qubits, full_input, crit_theta, crit_theta_par)
q_val = critic_qnode(full_input, crit_theta, crit_theta_par)

actor_qnode = make_actor_qnode(n_qubits)
#q_vals = actor_qnode(n_qubits, actor_inp_vec, theta, theta_par)
q_vals = actor_qnode(actor_inp_vec, theta, theta_par)

'''
# Forward pass
q_vals = actor_qnode(n_qubits, actor_inp_vec, theta, theta_par)
q_val = critic_qnode(n_crit_qubits, full_input, crit_theta, crit_theta_par)
'''

12
17


'\n# Forward pass\nq_vals = actor_qnode(n_qubits, actor_inp_vec, theta, theta_par)\nq_val = critic_qnode(n_crit_qubits, full_input, crit_theta, crit_theta_par)\n'

In [7]:
qml.draw_mpl(actor_qnode)(np.ones(n_qubits), np.ones(n_qubits), np.ones(n_qubits))
#qml.draw(actor_qnode)(n_qubits, np.ones(n_qubits), np.ones(n_qubits), np.ones(n_qubits))

In [6]:
qml.draw_mpl(critic_qnode)(np.ones(n_crit_qubits), np.ones(n_crit_qubits), np.ones(n_crit_qubits))

In [8]:
actor_grad_fn = qml.grad(actor_qnode, argnum=2)
grad_vals = actor_grad_fn(actor_inp_vec, theta, theta_par)
print(grad_vals)

[ 1.10475256e-18  2.20050497e-18 -1.04846713e-18  8.11175855e-19
  6.74153520e-19 -9.88522767e-19  7.89347303e-18 -4.97898671e-18
  1.62918291e-18 -6.53545727e-19  4.11124971e-17  1.67958585e-16]


In [12]:
critic_grad_fn = qml.grad(critic_qnode, argnum=2)
grad_vals = critic_grad_fn(full_input, crit_theta, crit_theta_par)
print(grad_vals)

[ 8.29961052e-20 -2.54388762e-19  4.08687470e-19  1.14778721e-18
  3.94974167e-19  8.09070693e-19 -5.04499809e-19  4.06237648e-19
 -5.83129637e-19  1.30463778e-19 -1.84157087e-19  1.77621797e-19
  8.92616715e-19 -1.27332201e-18 -2.91282869e-19 -2.17056644e-17
 -1.10347274e-16]


In [10]:
opt = qml.GradientDescentOptimizer(stepsize=0.1)
for _ in range(10):
    theta_par, loss = opt.step_and_cost(lambda tp: -critic_qnode(full_input, crit_theta, tp), crit_theta_par)
    print("Loss:", loss)

IndexError: index 12 is out of bounds for axis 0 with size 12

In [ ]:
# TODO: Implement updating the theta parameter for the RY gates 
# TODO: Implement updating the x input data vector for the RX gates based on the values stored in the Memory Experience Relay
#num_eps = 10
#for i in range(num_eps):
    